<!--start-block-->
<hr style="height: 1px;">
<i>This code was authored by the 8.S50x Course Team, Copyright 2021 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

# RECITATION 3: Fitting, `LMFIT` Software, and Likelihoods

<br>
<!--end-block--> 

<!--start-block-->
<hr style="height: 1px;">

## 3.0 Overview of Learning Objectives

In this recitation we will explore the following objectives:

- The principles of Frequentist fitting
- The software package `LMFIT`
- Likelihoods
- An `LMFIT` example more related to your project

The section on likelihoods also contains some practice with plotting distributions.

<br>
<!--end-block-->

<!--start-block-->
<hr style="height: 1px;">

## 3.1 Principles of Frequentist Fitting

Suppose you have some data $D$ you've collected, and a model for the data. Your model is really a function $f$ which maps from some model parameters $\theta$ to the data $D$. The purpose of the fit is to determine the true value of $\theta$ from the data.

In this language, $D$ and $\theta$ can be vectors or scalars depending on how many data points or parameters there are. To make things more complicated, the data points $D$ usually have an uncertainty on them called a <b>systematic uncertainty</b>. You can represent this by thinking of the data as a set of random variables.

(<i>Technical note:</i> In practice, thinking of the data as random variables means that the parameter $\theta$ is also a random variable. It therefore doesn't make sense to ask for the true <i>value</i> of $\theta$; instead we should ask for the true <i>distribution</i>. However, most people gloss over this fact by representing the "value" of $\theta$ as its distribution's mean.)


<!--end-block-->

<!--start-block-->

As you can imagine, the problem of finding the true $\theta$ is hard. That's why there are multiple fitting methods. The most popular are **Bayesian** and **frequentist** fitting, and we study frequentist here.

To fit a model function $f$ to data $D$ using the frequentist method, perform the following two steps.

1. Use your knowledge of the model to derive a <b>likelihood</b> function for your data. Likelihood is defined as the probability of the data $D$ occuring given some guess at parameters $\theta$. In the language of probability,
$$\mathcal{L}(\theta) = P(D | \theta)$$

2. Find the value $\theta_0$ which maximizes the likelihood. Take this value as your fit result.

That's it! In practice, you might also be interested in getting uncertainties on your fit result, so we'll discuss that in the next section.
<!--end-block-->

<!--start-block-->
#### <span style="color:purple">>>>QUESTION</span>


The frequentist method is so simple! Why do other methods exist? I.e., what are potential problems with the logic behind the method?

Hint: Why should the $\theta$ that maximizes likelihood be the true $\theta$? Should the goal be to maximize $P(D|\theta)$ or $P(\theta|D)$?

<br>
<!--end-block-->

<!--start-block-->
<hr style="height: 1px;">

## 3.3 Using `LMFIT` to fit to data

To get some practice fitting, suppose you have some data coming from the function $y=2x$.

<!--end-block-->

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Let's generate some example data with made up systematic uncertainties. These uncertainties are modeled as the standard deviations of normal distributions. Therefore, we draw each data point $y_i$ from a normal distribution with standard deviation equal to the uncertainty of point $i$ and mean $2 x_i$.

<!--
#initial code
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)
-->

<!--end-block-->

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Plot the data and its error bars.

<!--
#initial code
import matplotlib.pyplot as plt

#your code here
-->

<!--
#solution
import matplotlib.pyplot as plt

plt.errorbar(xi, yi, yerr=yerr, linestyle='none')
plt.scatter(xi, yi)
plt.xlabel("x")
plt.ylabel("y");
-->

In [ ]:
import matplotlib.pyplot as plt

plt.errorbar(xi, yi, yerr=yerr, linestyle='none')
plt.scatter(xi, yi)
plt.xlabel("x")
plt.ylabel("y");

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

We'll make a model using `LMFIT` to represent the data. Models can either be selected from a [large list of functions](https://lmfit.github.io/lmfit-py/builtin_models.html) already set up by `LMFIT`, or you can make them yourself. Here we use a preset linear model.

<!--
#initial code
from lmfit.models import LinearModel
model = LinearModel()
-->

<!--end-block-->

In [ ]:
from lmfit.models import LinearModel
model = LinearModel()

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

We'll skip the first set of fitting, using `LMFIT`'s automatic likelihood function (a chi-squared likelihood: see tomorrow's recitation). Let's jump straight to likelihood maximization. This maximization uses the same minimization methods you learned in class! Provide the data to `LMFIT`'s fit function.

<b>Important:</b> set the weights equal to one over the systematic uncertainty. Not the uncertainty itself, and not the variance.

<!--
#initial code
result = model.fit(yi, x=xi, weights=1/yerr);

print(result.fit_report())
-->

<!--end-block-->

In [ ]:
result = model.fit(yi, x=xi, weights=1/yerr);

print(result.fit_report())

<!--start-block-->

Look in the Variables section: we have a slope of about 2 and an intercept of about zero, consistent with the true model! Very helpfully, `LMFIT` also gives you uncertainties on the fit parameters. These result from propagating the uncertainties of the data set.

We'll talk about the chi-square and reduced chi-square statistics later.

<!--end-block-->

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

There's one more thing to do: it's always a good idea to verify that your model actually fits your data. So let's plot the data together with the model.

<!--
#initial code
result.plot();
-->

<!--end-block-->

In [ ]:
result.plot();

<!--start-block-->

You can see that the model fits the data very well in the bottom plot, and the top plot demonstrates that deviations of the data from the model are random; they don't seem correlated with $x$ nor with each other.


<!--end-block-->

<!--start-block-->
<hr style="height: 1px;">

## 3.3 Likelihood

To investigate the first step of the minimization process, let's look at a different example.

Suppose you're an astrophysicist looking at a distant star. Photons hit your telescope at random, independent intervals, so the the number that you detect within your period of observation is Poisson distributed.

Also, this star is really important, and $N\gg 1$ telescopes are looking at it. Your data $D$ is therefore is $\{n_1, n_2, \dots, n_N\}$ which is the counts observed by the telescopes during one day.

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Let's generate some Poisson-distributed sample data for each telescope. We'll assume a parameter of $\lambda=5$ counts per day, with $N=100$ telescopes.

<!--
#initial code
import numpy as np

LAMBDA = 5
N = 100

counts = np.random.poisson(LAMBDA, N);
-->

<!--end-block-->

In [ ]:
import numpy as np

LAMBDA = 5
N = 100

counts = np.random.poisson(LAMBDA, N);

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Since each telescope's detection $n_i$ is independent, the probability of detecting data set $D$ given some estimate of $\lambda$, which is the parameter, is simply the product of the probability for each telescope to detect $n_i$. This probability is Poisson distributed.

We would like to use `LMFIT`'s minimize function, which does not actually request the likelihood as an input. Instead, it asks for the logarithm of the likelihood of each data point, and assumes that each data point is independent. Internally, it adds all the likelihoods from all data points to get the log-likelihood of the data.

<!--
#initial code
import numpy as np
from scipy.stats import poisson

def log_likelihood(l, data):
    return np.log(poisson.pmf(data, l))
-->

In [ ]:
import numpy as np
from scipy.stats import poisson

def log_likelihood(l, data):
    return np.log(poisson.pmf(data, l))

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Use `LMFIT`'s minimize function to maximize the likelihood (i.e. minimize the negative likelihood) and recover our $\lambda=5$ value.

<!--
#initial code
from lmfit import minimize, fit_report

def negative_log_likelihood(l, data):
    return -log_likelihood(l, data)

params = Parameters()
params.add('l', min=0, value=1)

result = minimize(negative_log_likelihood, params, args=(counts,))
print(fit_report(result))
-->

<!--end-block-->

In [ ]:
from lmfit import Parameters, minimize, fit_report

def negative_log_likelihood(l, data):
    return -log_likelihood(l, data)

params = Parameters()
params.add('l', min=0, value=1)

result = minimize(negative_log_likelihood, params, args=(counts,))
print(fit_report(result))

<!--start-block-->
#### <span style="color:purple">>>>QUESTION</span>


Why is it okay to use the log likelihood instead of the likelihood for our likelihood function?

Hint: what is the one and only purpose of the likelihood function we have mentioned so far? How is it affected by taking the log?

<br>
<!--end-block-->

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Show the following in the same plot
* A histogram of the $n_i$ values observed by all the telescopes (choose your bins wisely)
* Superimposed, the best fit Poisson distribution with the $\lambda$ you got above (access the $\lambda$ with `result.params['l'].value` instead of copying it over)
* The true distribution ($\lambda$=5)
* Axis labels
* A legend

<!--
#initial code
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

#your code here
-->

<!--
#solution
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

bins = np.arange(np.max(counts)+1)-0.5
xs = bins+0.5
plt.hist(counts, bins=bins, label="Photon data", fill=False, histtype="step", color='k', linewidth=2)
plt.plot(xs, N * poisson.pmf(xs, result.params['l'].value), label="Best fit distro", color="C0", linewidth=2)
plt.plot(xs, N * poisson.pmf(xs,LAMBDA), label="True distro", color="C1", linewidth=2)

plt.xlabel("Number of photons observed")
plt.ylabel("Number of telescopes")
plt.legend()
-->

<!--end-block-->

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

bins = np.arange(np.max(counts)+1)-0.5
xs = bins+0.5
plt.hist(counts, bins=bins, label="Photon data", fill=False, histtype="step", color='k', linewidth=2)
plt.plot(xs, N * poisson.pmf(xs, result.params['l'].value), label="Best fit distro", color="C0", linewidth=2)
plt.plot(xs, N * poisson.pmf(xs,LAMBDA), label="True distro", color="C1", linewidth=2)

plt.xlabel("Number of photons observed")
plt.ylabel("Number of telescopes")
plt.legend()

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

To take into effect the uncertainty on $\lambda$, compute the Poisson distribution not just for the best fit value of $\lambda$, but also for $\lambda - 2\sigma_\lambda$, $\lambda+2\sigma_\lambda$, and ten $\lambda$s in between. Store those results in a list. Here, $\sigma_\lambda$ is the uncertainty on $\lambda$ generated by the fit and you can get it with `result.params['l'].stderr`.

Copy and paste your code from above and add an error band (`plt.fill_between`) where the lower edge of the error band in each bin represents the lowest number of telescopes predicted among all the distributions you computed above, and the higher edge represents the highest number of telescopes.

Please ask for help if you're confused about how to do this or about what you're being asked to do.

<!--
#initial code
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

#your code here
-->

<!--
#solution
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

bins = np.arange(np.max(counts)+1)-0.5
xs = bins+0.5
plt.hist(counts, bins=bins, label="Photon data", fill=False, histtype="step", color='k', linewidth=2)
plt.plot(xs, N * poisson.pmf(xs, result.params['l'].value), label="Best fit distro", color="C0", linewidth=2)
plt.plot(xs, N * poisson.pmf(xs,LAMBDA), label="True distro", color="C1", linewidth=2)

factor = np.linspace(-2, 2, 10)
distros = np.array([N * poisson.pmf(xs, result.params['l'].value + f * result.params['l'].stderr) for f in factor])
minimum = [np.min(distros[:,i]) for i in range(distros.shape[1])]
maximum = [np.max(distros[:,i]) for i in range(distros.shape[1])]
plt.fill_between(xs, minimum, maximum, alpha=0.5, color="C0")
plt.xlabel("Number of photons observed")
plt.ylabel("Number of telescopes")
plt.legend()
-->

<!--end-block-->

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

bins = np.arange(np.max(counts)+1)-0.5
xs = bins+0.5
plt.hist(counts, bins=bins, label="Photon data", fill=False, histtype="step", color='k', linewidth=2)
plt.plot(xs, N * poisson.pmf(xs, result.params['l'].value), label="Best fit distro", color="C0", linewidth=2)
plt.plot(xs, N * poisson.pmf(xs,LAMBDA), label="True distro", color="C1", linewidth=2)

factor = np.linspace(-2, 2, 10)
distros = np.array([N * poisson.pmf(xs, result.params['l'].value + f * result.params['l'].stderr) for f in factor])
minimum = [np.min(distros[:,i]) for i in range(distros.shape[1])]
maximum = [np.max(distros[:,i]) for i in range(distros.shape[1])]
plt.fill_between(xs, minimum, maximum, alpha=0.5, color="C0")
plt.xlabel("Number of photons observed")
plt.ylabel("Number of telescopes")
plt.legend()

<!--start-block-->

<hr style="height: 1px;">

## 3.4 A more relevant LMFIT example

Let's apply `LMFIT` in a setting more related to your project: we'll investigate a famous LIGO signal.

<!--end-block-->

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Download the signature of the first black hole detected (GW150914) from [the LIGO data archive]() and save it to the directory of this notebook. Then run this code.

<!--
#initial code
-->

<!--end-block-->

In [ ]:
from functions import gwfreq, osc_scale, osc, lower, higher
from gwpy.timeseries import TimeSeries
import h5py
from scipy.interpolate import interp1d

import os

fn = 'H-H1_GWOSC_16KHZ_R1-1126257415-4096.hdf5' # data file
tevent = 1126259462.422 # Mon Sep 14 09:50:45 GMT 2015
evtname = 'GW150914' # event name

detector = 'H1' # detecotr: L1 or H1

strain = TimeSeries.read(fn, format='hdf5.losc')
center = int(tevent)
strain = strain.crop(center-16, center+16)
asd = strain.asd(fftlength = 1)

NRtime, NR_H1 = np.genfromtxt('GW150914_4_NR_waveform.txt').transpose()
nrdata= TimeSeries(NR_H1, times = NRtime)

Define a function that whitens and filters the data.

In [ ]:
def whiten_and_bandpass(strain, asd, bp_low, bp_high):
    fft = strain.fft()
    asdinterp = interp1d(asd.frequencies, asd)
    asddiv =asdinterp(fft.frequencies)
    white_freq = fft/asddiv
    white = white_freq.ifft()
    whitebp = white.bandpass(bp_low,bp_high)
    return TimeSeries(whitebp, t0=strain.t0)

In [ ]:
low_pass = lower()
high_pass = higher()

whitened_data = whiten_and_bandpass(strain, asd, low_pass, high_pass)
wnr = whiten_and_bandpass(nrdata, asd, low_pass, high_pass).crop(-0.1, 0.05)
zoom = whitened_data.crop(tevent-0.09, tevent+0.05)
zoom.t0 = -0.09
x = np.array(zoom.times)
white_data_bp_zoom = zoom.value
plt.plot(x, white_data_bp_zoom)
plt.plot(wnr.times, wnr.value)

Define a funciton that...

In [ ]:
# define osc_dif for lmfit::minimize()
def osc_dif(params, x, data, data_asd, low_pass, high_pass):
    iM=params["Mc"]
    iT0=params["t0"]
    norm=params["C"]
    phi=params["phi"]
    model_strain = TimeSeries(osc(x, iM, iT0, phi, norm), times = x)
    val = whiten_and_bandpass(model_strain, data_asd, low_pass, high_pass)
    residuals = val-data
    return residuals

def osc_white(t, Mc, t0, phi, C, data_asd=None, low_pass = None, high_pass = None):
    model_strain = TimeSeries(osc(t, Mc, t0, phi, C), times = t)
    val = whiten_and_bandpass(model_strain, data_asd, low_pass, high_pass) 
    return val

Plot...

In [ ]:
owmodel = lmfit.Model(osc_white, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
owmodel.set_param_hint(name = 'Mc', value = 25, min = 10, max = 30)
owmodel.set_param_hint(name = 't0', value = 0.004, min = -0.015, max = 0.015)
owmodel.set_param_hint(name = 'C', value = 0.18853556, min = 0.01, max = 10)
owmodel.set_param_hint(name = 'phi', value = 1.24685424, min = 0, max = 2*np.pi)

omodel = lmfit.Model(osc_scale)
omodel.set_param_hint(name = 'Mc', value = 25)
omodel.set_param_hint(name = 't0', value = 0.01)
omodel.set_param_hint(name = 'C', value = 0.18853556)
omodel.set_param_hint(name = 'phi', value = 1.24685424)

result_white = owmodel.fit(white_data_bp_zoom, t = x)
print(result_white.fit_report())
result_white.plot(datafmt='-')

result = omodel.fit(white_data_bp_zoom, t = x)
print(result.fit_report())
result.plot(datafmt='-')



In [ ]:
import lmfit
from lmfit import Model, minimize, fit_report, Parameters

model = lmfit.Model(osc_white)
p = model.make_params()
p['Mc'].set(25)     # Mass guess
p['t0'].set(-0.005)  # By construction we put the merger in the center
p['C'].set(0.18853556)      # normalization guess
p['phi'].set(1.24685424)    # Phase guess
out = minimize(osc_dif, params=p, args=(x, white_data_bp_zoom, asd, low_pass, high_pass))
print(fit_report(out))
best_fit = model.eval(params=out.params,t=x, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
init_fit = model.eval(params=p,t=x, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
plt.plot(x, best_fit,'r',label='best fit')
#plt.plot(x, init_fit,'b',label='init')
plt.plot(x, white_data_bp_zoom)
plt.plot(wnr.times, wnr.value)
plt.show()

<!--start-block-->
#### <span style="color:purple">>>>QUESTION</span>


Question text needed...


<br>
<!--end-block-->